In [14]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.conditions import MaxMessageTermination


load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gemini-2.5-flash', api_key=api_key)

In [15]:
agent_1 = AssistantAgent(
    name='Writer_1',
    model_client=model_client,
    system_message="You are a helpful assistant. Give the output in less than 15 words",
)

agent_2 = AssistantAgent(
    name='Writer_2',
    model_client=model_client,
    system_message="You are a helpful assistant.Give the output in less than 15 words",
)

terminationCondition = MaxMessageTermination(max_messages=3)

agent_team = RoundRobinGroupChat(participants=[agent_1, agent_2],termination_condition=terminationCondition)

In [16]:
from autogen_agentchat.ui import Console

stream = agent_team.run_stream(task="Write a poem about teddy in 4 lines")

await Console(stream)

---------- TextMessage (user) ----------
Write a poem about teddy in 4 lines


---------- TextMessage (Writer_1) ----------
My soft friend,
Always near.
Chases fright,
Holds me dear.
---------- TextMessage (Writer_2) ----------
Soft, cuddly friend,
Always by my side.
Comfort and joy,
My favorite hide.


TaskResult(messages=[TextMessage(id='d7fd9c57-5020-4bb5-a1e6-bb88cea718d3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 26, 38, 282477, tzinfo=datetime.timezone.utc), content='Write a poem about teddy in 4 lines', type='TextMessage'), TextMessage(id='60971453-c176-435d-b93a-276539c40126', source='Writer_1', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 26, 42, 61045, tzinfo=datetime.timezone.utc), content='My soft friend,\nAlways near.\nChases fright,\nHolds me dear.', type='TextMessage'), TextMessage(id='bc1f299e-408c-4d3a-9318-c6d543f5696e', source='Writer_2', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 26, 43, 539938, tzinfo=datetime.timezone.utc), content='Soft, cuddly friend,\nAlways by my side.\nComfort and joy,\nMy favorite hide.', type='TextMessage')], stop_reason='Maxim

#### Save the Team State

In [17]:
team_state = await agent_team.save_state()
print("Team state saved.")
print(team_state)

Team state saved.
{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'Writer_1': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a poem about teddy in 4 lines', 'source': 'user', 'type': 'UserMessage'}, {'content': 'My soft friend,\nAlways near.\nChases fright,\nHolds me dear.', 'thought': None, 'source': 'Writer_1', 'type': 'AssistantMessage'}]}}, 'message_buffer': [{'id': 'bc1f299e-408c-4d3a-9318-c6d543f5696e', 'source': 'Writer_2', 'models_usage': {'prompt_tokens': 46, 'completion_tokens': 22}, 'metadata': {}, 'created_at': datetime.datetime(2025, 7, 25, 18, 26, 43, 539938, tzinfo=datetime.timezone.utc), 'content': 'Soft, cuddly friend,\nAlways by my side.\nComfort and joy,\nMy favorite hide.', 'type': 'TextMessage'}]}, 'Writer_2': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', '

#### Reset the Team

In [18]:
await agent_team.reset()

#### Let's test

In [19]:
stream = agent_team.run_stream(task="What was the last line of Poem you wrote?")

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of Poem you wrote?
---------- TextMessage (Writer_1) ----------
As an AI, I don't write poems in a personal sense. I generate text.
---------- TextMessage (Writer_2) ----------
As an AI, I don't write poems personally. I generate text. No last line exists.


TaskResult(messages=[TextMessage(id='93f12399-b3aa-48b3-943a-ab0f8e05345c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 26, 55, 332727, tzinfo=datetime.timezone.utc), content='What was the last line of Poem you wrote?', type='TextMessage'), TextMessage(id='58df62f7-fd50-4f41-9d21-5dce06befafe', source='Writer_1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 26, 58, 341730, tzinfo=datetime.timezone.utc), content="As an AI, I don't write poems in a personal sense. I generate text.", type='TextMessage'), TextMessage(id='2c212ac7-239f-47d2-b2a7-1dfc734f8dcc', source='Writer_2', models_usage=RequestUsage(prompt_tokens=48, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 27, 0, 277911, tzinfo=datetime.timezone.utc), content="As an AI, I don't write poems personally. I generate text. No last line exists.", type='TextMessage')], stop

Since it got reset. It doesn't have anything.

#### Load and check

In [20]:
await agent_team.load_state(team_state)
print("Loaded state successfully")

stream = agent_team.run_stream(task="What was the last line of Poem you wrote?")
await Console(stream)

Loaded state successfully
---------- TextMessage (user) ----------
What was the last line of Poem you wrote?
---------- TextMessage (Writer_1) ----------
Holds me dear.
---------- TextMessage (Writer_2) ----------
My favorite hide.


TaskResult(messages=[TextMessage(id='f0f8bde1-9a4a-48a8-b77b-380e5064bded', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 27, 3, 947802, tzinfo=datetime.timezone.utc), content='What was the last line of Poem you wrote?', type='TextMessage'), TextMessage(id='3a2e4cc3-c306-4f08-bba5-5b2bb9942ac5', source='Writer_1', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 27, 5, 140809, tzinfo=datetime.timezone.utc), content='Holds me dear.', type='TextMessage'), TextMessage(id='b0928448-c15c-4e51-9ef9-20c9ffe0ec47', source='Writer_2', models_usage=RequestUsage(prompt_tokens=85, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 7, 25, 18, 27, 5, 916712, tzinfo=datetime.timezone.utc), content='My favorite hide.', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')